# BERTopic

In [1]:
from bertopic import BERTopic
import pandas as pd
import numpy as np
import tqdm as notebook_tqdm

C:\Users\johan\anaconda3\envs\text_mining\lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\johan\anaconda3\envs\text_mining\lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\johan\anaconda3\envs\text_mining\lib\site-packages\umap\distances.py:

In [2]:
df = pd.read_parquet("frankenthal.parquet")
df_filtered = df[df.doc_typ == ('Vorlage' or 'Beschluss' or 'Antrag' or 'Anfrage')]
df_filtered = df_filtered.drop(['status', 'top_name', 'ocr', 'rel_path_to_file'], axis=1)

In [3]:
df_verkehr = df_filtered[df_filtered['verkehrsbezug']==True]

In [4]:
topic_model = BERTopic(language="german", nr_topics="auto")

In [5]:
topics, probs = topic_model.fit_transform(df_filtered['content'])

In [6]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,364,-1_der_die_und_des,"[der, die, und, des, in, für, im, den, von, zu]",[ \nBeratungsergebnis: \n \nGremium \nSitzung ...
1,0,1807,0_der_die_und_in,"[der, die, und, in, des, für, den, im, mit, von]",[ \nBeratungsergebnis: \n \nGremium \nSitzung ...
2,1,210,1_cdu_stadtratsfraktion_stimmen_stadtrat,"[cdu, stadtratsfraktion, stimmen, stadtrat, de...",[ \nBeratungsergebnis: \n \nGremium \nSitzung ...
3,2,158,2_spd_stadtratsfraktion_der_stadtrat,"[spd, stadtratsfraktion, der, stadtrat, stimme...",[ \nBeratungsergebnis: \n \nGremium \nSitzung ...
4,3,71,3_opel_ratsmitglied_xv_von,"[opel, ratsmitglied, xv, von, stimmen, stadtra...",[ \nBeratungsergebnis: \n \nGremium \nSitzung ...
5,4,22,4_der_die_in_für,"[der, die, in, für, von, und, im, den, bis, fr...",[ \nBeratungsergebnis: \n \nGremium \nSitzung ...
6,5,19,5_der_in_die_im,"[der, in, die, im, teur, von, und, sich, für, ...",[ \nBeratungsergebnis: \n \nGremium \nSitzung ...
7,6,17,6_in_kinder_der_die,"[in, kinder, der, die, plätze, 01, 12, für, un...",[ \nBeratungsergebnis: \n \nGremium \nSitzung ...
8,7,15,7_der_die_und_in,"[der, die, und, in, stadtklinik, eur, den, mit...",[ \nBeratungsergebnis: \n \nGremium \nSitzung ...


In [7]:
topic_model.get_topic(6)

[('in', 0.060463187356439047),
 ('kinder', 0.04910993485206752),
 ('der', 0.047079784005678575),
 ('die', 0.04325919557868196),
 ('plätze', 0.040117381274462904),
 ('01', 0.03288487216478792),
 ('12', 0.032518412224845486),
 ('für', 0.031349845662882445),
 ('und', 0.02912207431391012),
 ('mit', 0.025450842710952083)]

## Erkenntnisse
- Modell scheint nicht gut auf die deutsche Sprache angepasst zu sein.
- Es werden viele sog. Stoppwörter als Thema erkannt
- Der Wiederkehrende Kopf der Dokumente scheint ein Problem darzustellen

# NLTK

In [9]:
import nltk 
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stopwords=set(stopwords.words('german'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\johan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\johan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
import plotly

In [10]:
# Remove stopwords from an array of strings
def remove_stopwords(data):
    output_array=[]
    for sentence in data:
        temp_list=[]
        for word in str(sentence).split():
            if word.lower() not in stopwords:
                temp_list.append(word)
        output_array.append(' '.join(temp_list))
    return output_array

In [11]:
verkehr_nostop = remove_stopwords(df_verkehr['content'])

In [23]:
# Create model based on German BERT
# model = BERTopic(embedding_model=bertGerman, language="german")
model = BERTopic(language="german", nr_topics="auto")

In [25]:
topics, probabilities = model.fit_transform(verkehr_nostop)

In [26]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,9,-1_m²_lfdm_00_je,"[m², lfdm, 00, je, laufbahn, autoskooter, vers...",[Beratungsergebnis: Gremium Sitzung Top Öffent...
1,0,592,0_frankenthal_pfalz_stadt_werden,"[frankenthal, pfalz, stadt, werden, 00, 000, e...",[Beratungsergebnis: Gremium Sitzung Top Öffent...
2,1,80,1_straße_frankenthal_sowie_werden,"[straße, frankenthal, sowie, werden, stadt, ba...",[Beratungsergebnis: Gremium Sitzung Top Öffent...
3,2,41,2_anlage_xiv_baugb_frankenthal,"[anlage, xiv, baugb, frankenthal, bebauungspla...",[Beratungsergebnis: Gremium Sitzung Top Öffent...
4,3,31,3_stadtklinik_frankenthal_höhe_pfalz,"[stadtklinik, frankenthal, höhe, pfalz, eur, 2...",[Beratungsergebnis: Gremium Sitzung Top Öffent...
5,4,30,4_spd_stadtratsfraktion_stadtrat_stimmen,"[spd, stadtratsfraktion, stadtrat, stimmen, fr...",[Beratungsergebnis: Gremium Sitzung Top Öffent...
6,5,18,5_cdu_stadtratsfraktion_stadtrat_stimmen,"[cdu, stadtratsfraktion, stadtrat, stimmen, hi...",[Beratungsergebnis: Gremium Sitzung Top Öffent...
7,6,17,6_stufe_dm_00_abs,"[stufe, dm, 00, abs, iii, ii, anlage, baugb, 5...",[Beratungsergebnis: Gremium Sitzung Top Öffent...
8,7,15,7_nr_flurstück_cm_straße,"[nr, flurstück, cm, straße, grundstücks, foltz...",[Beratungsergebnis: Gremium Sitzung Top Öffent...


In [32]:
model.get_topic(3)

[('stadtklinik', 0.03690397977792315),
 ('frankenthal', 0.030592382905206782),
 ('höhe', 0.02411694415842366),
 ('pfalz', 0.020442105431458274),
 ('eur', 0.020363251684974482),
 ('2007', 0.01901279926109363),
 ('teur', 0.017959198172514502),
 ('jahren', 0.0179524737702795),
 ('kinder', 0.01738527761386893),
 ('krankenhausausschuss', 0.01657168093249978)]

In [33]:
model.update_topics(verkehr_nostop, n_gram_range=(1, 3))

In [34]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,9,-1_m²_lfdm_00_je,"[m², lfdm, 00, je, laufbahn, anlage, werden, f...",[Beratungsergebnis: Gremium Sitzung Top Öffent...
1,0,592,0_frankenthal_pfalz_stadt_frankenthal pfalz,"[frankenthal, pfalz, stadt, frankenthal pfalz,...",[Beratungsergebnis: Gremium Sitzung Top Öffent...
2,1,80,1_straße_frankenthal_sowie_werden,"[straße, frankenthal, sowie, werden, stadt, an...",[Beratungsergebnis: Gremium Sitzung Top Öffent...
3,2,41,2_anlage_xiv_frankenthal_baugb,"[anlage, xiv, frankenthal, baugb, abs, bebauun...",[Beratungsergebnis: Gremium Sitzung Top Öffent...
4,3,31,3_stadtklinik_frankenthal_stadtklinik frankent...,"[stadtklinik, frankenthal, stadtklinik franken...",[Beratungsergebnis: Gremium Sitzung Top Öffent...
5,4,30,4_spd stadtratsfraktion_spd_stadtratsfraktion_...,"[spd stadtratsfraktion, spd, stadtratsfraktion...",[Beratungsergebnis: Gremium Sitzung Top Öffent...
6,5,18,5_cdu_cdu stadtratsfraktion_stadtrat_stadtrats...,"[cdu, cdu stadtratsfraktion, stadtrat, stadtra...",[Beratungsergebnis: Gremium Sitzung Top Öffent...
7,6,17,6_stufe_dm_00_00 dm,"[stufe, dm, 00, 00 dm, abs, iii, stufe ii, stu...",[Beratungsergebnis: Gremium Sitzung Top Öffent...
8,7,15,7_nr_flurstück nr_flurstück_straße,"[nr, flurstück nr, flurstück, straße, frankent...",[Beratungsergebnis: Gremium Sitzung Top Öffent...


## Erkentnisse
- Ergebnisse sind sehr unkonkret
- Die Klinik scheint ein Thema gewesen zu sein.
- Bauen war auch ein Thema

# TODO
- Visualisierungen funktionieren noch nicht
- Methodik evtl verbessern